<a href="https://colab.research.google.com/github/rsaran-BioAI/AGILE/blob/main/SMILES_GraphRep_VAE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Connecting the drive with Colab

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install rdkit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.5/30.5 MB 42.3 MB/s eta 0:00:00


In [8]:
from rdkit import Chem
from rdkit.Chem import Draw
import torch
import numpy as np
import pandas as pd

In [17]:
!pip install torch_geometric

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 7.5 MB/s eta 0:00:00


In [18]:
from torch_geometric.data import Data

In [9]:
# Load the spreadsheet
file_path = '/content/drive/MyDrive/AGILE2/SMILES_features.csv'
smiles_data = pd.read_csv(file_path)

In [10]:
# Extracting SMILES strings
smiles_list = smiles_data['smiles'].tolist()

In [11]:
# Display the first few rows to understand the data
smiles_data.head()

,smiles,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 353,Unnamed: 354,Unnamed: 355,Unnamed: 356,Unnamed: 357,Unnamed: 358,Unnamed: 359,Unnamed: 360,Unnamed: 361,Unnamed: 362
0,CCCCCCCC\C=C/CCCCCCCCNC(=O)C(CCCCCOC(=O)CCCCCC...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,CCCCCCCCCCCCCCCCCCNC(=O)C(CCCCCOC(=O)CCCCCCC\C...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,CCCCCCCCCCCCNC(=O)C(CCCCCOC(=O)CCCCCCC\C=C/CCC...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,CCCCCCCCCCCCCCNC(=O)C(CCCCCOC(=O)CCCCCCC\C=C/C...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,CCCCCCCCCCCCCCCCNC(=O)C(CCCCCOC(=O)CCCCCCC\C=C...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
# Extracting SMILES strings
smiles_list = smiles_data['smiles'].tolist()

In [13]:
# Printing the number of SMILES strings in the list
print(len(smiles_list))

1200


In [19]:
def smiles_to_graph(smiles):
    mol = Chem.MolFromSmiles(smiles)
    atoms = [atom.GetAtomicNum() for atom in mol.GetAtoms()]
    nodes = torch.tensor(atoms, dtype=torch.float).view(-1, 1)

    edge_index = []
    edge_attr = []
    for bond in mol.GetBonds():
        start, end = bond.GetBeginAtomIdx(), bond.GetEndAtomIdx()
        edge_index.append((start, end))
        edge_index.append((end, start))  # add both directions for undirected graph
        # You can add more bond features here

    edge_index = torch.tensor(edge_index, dtype=torch.long).t().contiguous()
    return Data(x=nodes, edge_index=edge_index)

In [20]:
graphs = [smiles_to_graph(smiles) for smiles in smiles_list]

In [23]:
print(graphs)

[Data(x=[57, 1], edge_index=[2, 112]), Data(x=[57, 1], edge_index=[2, 112]), Data(x=[51, 1], edge_index=[2, 100]), Data(x=[53, 1], edge_index=[2, 104]), Data(x=[55, 1], edge_index=[2, 108]), Data(x=[56, 1], edge_index=[2, 112]), Data(x=[56, 1], edge_index=[2, 112]), Data(x=[50, 1], edge_index=[2, 100]), Data(x=[52, 1], edge_index=[2, 104]), Data(x=[54, 1], edge_index=[2, 108]), Data(x=[56, 1], edge_index=[2, 112]), Data(x=[56, 1], edge_index=[2, 112]), Data(x=[50, 1], edge_index=[2, 100]), Data(x=[52, 1], edge_index=[2, 104]), Data(x=[54, 1], edge_index=[2, 108]), Data(x=[54, 1], edge_index=[2, 108]), Data(x=[54, 1], edge_index=[2, 108]), Data(x=[48, 1], edge_index=[2, 96]), Data(x=[50, 1], edge_index=[2, 100]), Data(x=[52, 1], edge_index=[2, 104]), Data(x=[53, 1], edge_index=[2, 104]), Data(x=[53, 1], edge_index=[2, 104]), Data(x=[47, 1], edge_index=[2, 92]), Data(x=[49, 1], edge_index=[2, 96]), Data(x=[51, 1], edge_index=[2, 100]), Data(x=[54, 1], edge_index=[2, 106]), Data(x=[54, 1]

In [25]:
def find_max_nodes(graphs):
    max_nodes = 0
    for graph in graphs:
        if graph.num_nodes > max_nodes:
            max_nodes = graph.num_nodes
    return max_nodes

# Assuming 'graphs' is your list of graph data
max_nodes = find_max_nodes(graphs)
print(f"The maximum number of nodes in any graph is: {max_nodes}")

The maximum number of nodes in any graph is: 60


In [27]:
def find_max_edges(graphs):
    max_edges = 0
    for graph in graphs:
        num_edges = graph.edge_index.size(1)  # Number of columns in edge_index
        if num_edges > max_edges:
            max_edges = num_edges
    return max_edges

# Assuming 'graphs' is your list of graph data
max_edges = find_max_edges(graphs)
print(f"The maximum number of edges in any graph is: {max_edges}")

The maximum number of edges in any graph is: 118


In [26]:
import torch
from torch_geometric.data import Data

def pad_graph(graph, max_nodes, max_edges):
    num_nodes = graph.num_nodes
    num_edges = graph.edge_index.size(1)

    # Pad node features
    node_padding = max_nodes - num_nodes
    padded_x = torch.cat([graph.x, torch.zeros(node_padding, graph.x.size(1))], dim=0)

    # Pad edge index
    edge_padding = max_edges - num_edges
    dummy_edges = torch.tensor([[num_nodes, num_nodes]] * edge_padding).t()
    padded_edge_index = torch.cat([graph.edge_index, dummy_edges], dim=1)

    return Data(x=padded_x, edge_index=padded_edge_index)

In [28]:
padded_graphs = [pad_graph(graph, max_nodes, max_edges) for graph in graphs]

In [30]:
import torch
from torch.utils.data import Dataset, DataLoader

class GraphDataset(Dataset):
    def __init__(self, graphs):
        self.graphs = graphs

    def __len__(self):
        return len(self.graphs)

    def __getitem__(self, idx):
        graph = self.graphs[idx]
        return graph.x, graph.edge_index

# Convert your list of padded graphs to a dataset
graph_dataset = GraphDataset(padded_graphs)

# Split the dataset into training and testing
train_size = int(0.8 * len(graph_dataset))
test_size = len(graph_dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(graph_dataset, [train_size, test_size])

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [35]:
print(len(graph_dataset))
print(len(test_dataset))
print(len(train_dataset))

1200
240
960
